<a href="https://colab.research.google.com/github/xusanduo7399-hue/VA-eval/blob/main/LLMeval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers accelerate bitsandbytes einops


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'mistralai/Mistral-7B-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    load_in_4bit=True,
)

text = 'User: Hello, how are you'
inputs = tokenizer(text, return_tensors='pt').to('cuda')

tokenizer.pad_token = tokenizer.eos_token

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


User: Hello, how are you?

Tarik: I'm good.

User: Good. Great.

Tarik: You?

User: Yeah, I'm good.

Tarik: Good.

User: You're a teacher, right?

Tarik: Yeah, I'm a teacher.

User: Yeah, you're a teacher?

Tarik: Yeah.

User: Teacher?


In [ ]:
import pandas as pd
df = pd.read_csv("/content/blender90_preds.csv", encoding="latin1", engine="python")
df.head()


,text
0,Tell me about the movie Star Wars.
1,Suggest ways to reduce daily water use by hou...
2,"Classify the following movie genres: comedy, ..."
3,Calculate the area of a triangle with length ...
4,Change the subject of the following sentence:...


In [ ]:
results = []

for idx, row in df.iterrows():
    question = row["text"]

    prompt = f"User: {question}\nAssistant:"

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to("cuda")

    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        inputs.input_ids,
        attention_mask=attention_mask,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    if "Assistant:" in answer:
        answer = answer.split("Assistant:")[-1].strip()

    results.append(answer)

df["model_answer"] = results

df.to_csv("mistral_results.csv", index=False)
df.head()


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,text,model_answer
0,Tell me about the movie Star Wars.,It’s about a young rebel fighter named Luke Sk...
1,Suggest ways to reduce daily water use by hou...,"To reduce water use in the kitchen, avoid runn..."
2,"Classify the following movie genres: comedy, ...",San Jose
3,Calculate the area of a triangle with length ...,The area of a triangle with length
4,Change the subject of the following sentence:...,The cat meowed.\n\nThe cat meowed.\n\nThe cat ...


In [ ]:
import pandas as pd

df = pd.read_csv("/content/mistral_results.csv", encoding="utf-8", engine="python")

# 转成 Windows 友好的 UTF-8-SIG（带 BOM）
df.to_csv("/content/mistral_resultsutf8.csv", index=False, encoding="utf-8-sig")
